# <p style="text-align: center;"> Variational Linear Systems Code </p> 
<p style="text-align: center;"> Ryan LaRose </p>

This notebook briefly demonstrates the current state of the Variational Linear Systems (VLS) code. All code is contained in `vls_pauli.py`, which defines a `PauliSystem` class.

In [47]:
# imports
from vls_pauli import PauliSystem

import numpy as np

# Creating a Linear System of Equations

A `PauliSystem` consists of a matrix of the form
\begin{equation}
A = \sum_{k = 1}^{K} c_k \sigma_k
\end{equation}
where $c_k$ are complex coefficients and $\sigma_k$ are strings of Pauli operators. In code, we represent the matrix $A$ as arrays of strings corresponding to Pauli operators. For example, to represent the Pauli operators
\begin{align}
\sigma_1 &= \sigma_X \otimes \sigma_Y \otimes \sigma_X \\
\sigma_2 &= \sigma_Z \otimes \sigma_X \otimes \sigma_I 
\end{align}
we would write:

In [48]:
Amat_ops = np.array([["X", "Y", "X", "Z"],
                     ["Z", "X", "I", "Y"]])

Coefficients $c_k$ are stored similarly as arrays of complex values:

In [49]:
Amat_coeffs = np.array([0.5 + 0.5j, 1 - 2j])

Finally, the solution vector
\begin{equation}
|b\rangle = U |0\>
\end{equation}
is represented by the unitary $U$ that (efficiently) prepares $|b\rangle$ from the ground state. For example, the unitary $U$ could be
\begin{equation}
U = \sigma_x \otimes \sigma_x \otimes \sigma_I,
\end{equation}
which we would represent in code as:

In [50]:
Umat_ops = np.array(["X", "X", "I", "X"])

To create `PauliSystem`, we can then simply feed in `Amat_coeffs`, `Amat_ops`, and `Umat_ops`.

In [51]:
system = PauliSystem(Amat_coeffs, Amat_ops, Umat_ops)

# Working with a Pauli System

To see the actual matrix representation of the system (in the computational basis), we can do:

In [52]:
matrix = system.matrix()
print("Size of matrix:", matrix.shape)
print(matrix)

Size of matrix: (16, 16)
[[ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j  -2. -1.j   0. +0.j
   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0.5-0.5j  0. +0.j ]
 [ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   2. +1.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j  -0.5+0.5j]
 [ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
  -2. -1.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0.5-0.5j  0. +0.j
   0. +0.j   0. +0.j ]
 [ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   2. +1.j
   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j  -0.5+0.5j
   0. +0.j   0. +0.j ]
 [ 0. +0.j  -2. -1.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j   0. +0.j  -0.5+0.5j  0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j ]
 [ 2. +1.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0.5-0.5j  0. +0.j   0. +0.j
   0. +

We can also see the solution vector $|b\rangle$ by doing:

In [53]:
b = system.vector()

AttributeError: 'PauliSystem' object has no attribute 'vector'

# Creating an Ansatz

Initially, the `PauliSystem` ansatz for $V$ is an empty circuit:

In [24]:
print(system.ansatz)

We are free to pick whatever ansatz we wish. The method `PauliSystem.make_ansatz_circuit` is currently set to make an alternating two-qubit gate ansatz.

In [26]:
system.make_ansatz_circuit()
print(system.ansatz)

1: ───X^Symbol("0")────Y^Symbol("1")────Z^Symbol("2")────@───X^Symbol("6")───Y^Symbol("7")────Z^Symbol("8")────@─────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                         │                                                     │
2: ───X^Symbol("3")────Y^Symbol("4")────Z^Symbol("5")────X───X^Symbol("9")───Y^Symbol("10")───Z^Symbol("11")───X───X^Symbol("12")───Y^Symbol("13")───Z^Symbol("14")───@───X^Symbol("18")───Y^Symbol("19")───Z^Symbol("20")───@───
                                                                                                                                                                      │                                                      │
3: ───X^Symbol("15")───Y^Symbol("16")───Z^Symbol("17")────────────────────────────────────────────────────────────────────────────────────────────────────────────────X───X^Symbol("21")───Y^Symbol("22")───Z^Symbol

This circuit contains 48 parameters (4 qubits x 2 "gates" / qubit x 6 parameters / gate). (Note that printing the circuit gets cut off in the notebook.) For our simple example, we will chop off some of the gates to make the optimization easier:

In [27]:
system.ansatz = system.ansatz[:-9]
print(system.ansatz)

1: ───X^Symbol("0")────Y^Symbol("1")────Z^Symbol("2")────@───X^Symbol("6")───Y^Symbol("7")────Z^Symbol("8")────
                                                         │
2: ───X^Symbol("3")────Y^Symbol("4")────Z^Symbol("5")────X───X^Symbol("9")───Y^Symbol("10")───Z^Symbol("11")───

3: ───X^Symbol("15")───Y^Symbol("16")───Z^Symbol("17")─────────────────────────────────────────────────────────
